In [ ]:
team ID -3
semester-5
division-B
school-CSE
Topic ID-5
PROJECT TITLE-Pump it Up:Data Mining The Water Table
Team Leader-Krutika M K
Team members- 1)Shriya hireholi r.no-79  USN-01FE16BCS079
              2)Kartik R D      r.no-89  USN-01FE16BCS089
              3)Krutika M K     r.no-94  USN-01FE16BCS094
              4)Mahim Patil     r.no-97  USN-01FE16BCS097


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#load the training data
train_values = pd.read_csv('training_set_values.csv')

In [4]:
#load the training set labels
train_labels = pd.read_csv('training_set_labels.csv')

In [5]:
#combine training and the labels for it
combined = train_values.merge(train_labels, on='id')

In [6]:
#check it worked
print(combined.head())
print(combined.shape)

      id  amount_tsh date_recorded        funder  gps_height     installer  \
0  69572      6000.0    2011-03-14         Roman        1390         Roman   
1   8776         0.0    2013-03-06       Grumeti        1399       GRUMETI   
2  34310        25.0    2013-02-25  Lottery Club         686  World vision   
3  67743         0.0    2013-01-28        Unicef         263        UNICEF   
4  19728         0.0    2011-07-13   Action In A           0       Artisan   

   longitude   latitude              wpt_name  num_private       ...        \
0  34.938093  -9.856322                  none            0       ...         
1  34.698766  -2.147466              Zahanati            0       ...         
2  37.460664  -3.821329           Kwa Mahundi            0       ...         
3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0       ...         
4  31.130847  -1.825359               Shuleni            0       ...         

  water_quality quality_group      quantity  quantity_group  \

In [7]:
#start to remove unnecessary features

In [8]:
#recorded_by has no unique values
combined['recorded_by'].unique()

array(['GeoData Consultants Ltd'], dtype=object)

In [9]:
combined.drop('recorded_by', axis=1, inplace=True)

In [10]:
c = combined
c = c[c.amount_tsh==0]
len(c)

41639

In [11]:
#dropping this column because too many missing values
combined.drop('amount_tsh', axis=1, inplace=True)

In [12]:
#doing the same with public meeting and permit columns
combined.drop('public_meeting', axis=1, inplace=True)
combined.drop('permit', axis=1, inplace=True)

In [13]:
#look at categorical predictors that show similar information
#start with extraction_type, extraction_type_group, and extraction_type_class

d = combined
d.groupby(['extraction_type', 'extraction_type_group', 'extraction_type_class']).size()

extraction_type            extraction_type_group  extraction_type_class
afridev                    afridev                handpump                  1770
cemo                       other motorpump        motorpump                   90
climax                     other motorpump        motorpump                   32
gravity                    gravity                gravity                  26780
india mark ii              india mark ii          handpump                  2400
india mark iii             india mark iii         handpump                    98
ksb                        submersible            submersible               1415
mono                       mono                   motorpump                 2865
nira/tanira                nira/tanira            handpump                  8154
other                      other                  other                     6430
other - mkulima/shinyanga  other handpump         handpump                     2
other - play pump          other hand

In [14]:
# Group low occuring counts together or into related groups
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

combined['extraction_type'][combined.extraction_type.isin(['india mark ii','india mark iii'])] = 'india mark'
combined['extraction_type_class'][combined.extraction_type_class.isin(['india mark ii','india mark iii'])] = 'india mark'
combined['extraction_type'][combined.extraction_type.isin(['cemo', 'climax'])] = 'motorpump'
combined['extraction_type_class'][combined.extraction_type_class.isin(['cemo', 'climax'])] = 'motorpump'
combined['extraction_type'][combined.extraction_type.isin(['other - mkulima/shinyanga','other - play pump', 'walimi'])] = 'handpump'
combined['extraction_type_class'][combined.extraction_type_class.isin(['other - mkulima/shinyanga','other - play pump', 'walimi'])] = 'handpump'
combined['extraction_type'][combined.extraction_type.isin(['other - swn 81','swn 80'])] = 'swn'
combined['extraction_type_class'][combined.extraction_type_class.isin(['other - swn 81','swn 80'])] = 'swn'


In [15]:
#check it worked
d = combined
d.groupby(['extraction_type', 'extraction_type_group', 'extraction_type_class']).size()                       

extraction_type    extraction_type_group  extraction_type_class
afridev            afridev                handpump                  1770
gravity            gravity                gravity                  26780
handpump           other handpump         handpump                   135
india mark         india mark ii          handpump                  2400
                   india mark iii         handpump                    98
ksb                submersible            submersible               1415
mono               mono                   motorpump                 2865
motorpump          other motorpump        motorpump                  122
nira/tanira        nira/tanira            handpump                  8154
other              other                  other                     6430
other - rope pump  rope pump              rope pump                  451
submersible        submersible            submersible               4764
swn                other handpump         handpump          

In [16]:
#remove extraction_type_group
combined.drop('extraction_type_group', axis=1, inplace=True)

In [17]:
d = combined
d.groupby(['management', 'management_group']).size()      

management        management_group
company           commercial            685
other             other                 844
other - school    other                  99
parastatal        parastatal           1768
private operator  commercial           1971
trust             commercial             78
unknown           unknown               561
vwc               user-group          40507
water authority   commercial            904
water board       user-group           2933
wua               user-group           2535
wug               user-group           6515
dtype: int64

In [18]:
d = combined
d.groupby(['scheme_management', 'scheme_name']).size() 

scheme_management  scheme_name                 
Company            BFFS                              1
                   Bagamoyo Wate                    60
                   Bagamoyo wate                   210
                   Borehole                          1
                   Borehole drilling project         1
                   Bulamba piped scheme              2
                   Burieni water supply              1
                   Doe water supply                  3
                   EKTM 2 water  supply              1
                   EKTM 2 water suply               25
                   EKTM 2 water supply               9
                   EKTM 3 water supply              51
                   Fufulamsuri water supply          1
                   GRUMETI                           6
                   HESAWA                            1
                   Hakwe water supply               10
                   Holili water supply              23
                 

In [19]:
#remove scheme_name because too many different scheme types
combined.drop('scheme_name', axis=1, inplace=True)

In [66]:
combined['scheme_management']

0                     vwc
1                   other
2                     vwc
3                     vwc
4                   other
5                     vwc
6                     vwc
7                   other
8                     vwc
9                   other
10       private operator
11                    wug
12                    wug
13            water board
14                    wug
15                    vwc
16                    wua
17                    wua
18                  other
19                    vwc
20                    vwc
21                    wug
22                    wua
23        water authority
24                    vwc
25                    vwc
26                    vwc
27                  other
28                company
29            water board
               ...       
59370               other
59371                 wug
59372    private operator
59373                 vwc
59374                 wug
59375                 vwc
59376                 vwc
59377     wa

In [20]:
d = combined
d.groupby(['payment', 'payment_type']).size() 

payment                payment_type
never pay              never pay       25348
other                  other            1054
pay annually           annually         3642
pay monthly            monthly          8300
pay per bucket         per bucket       8985
pay when scheme fails  on failure       3914
unknown                unknown          8157
dtype: int64

In [21]:
# these look the same so we can drop payment
combined.drop('payment', axis=1, inplace=True)

In [22]:
d = combined
d.groupby(['water_quality', 'quality_group']).size() 

water_quality       quality_group
coloured            colored            490
fluoride            fluoride           200
fluoride abandoned  fluoride            17
milky               milky              804
salty               salty             4856
salty abandoned     salty              339
soft                good             50818
unknown             unknown           1876
dtype: int64

In [23]:
#these look the same as well 
#keep the one with more description
combined.drop('quality_group', axis=1, inplace=True)

In [24]:
d = combined
d.groupby(['quantity', 'quantity_group']).size() 

quantity      quantity_group
dry           dry                6246
enough        enough            33186
insufficient  insufficient      15129
seasonal      seasonal           4050
unknown       unknown             789
dtype: int64

In [25]:
#these look the same as well 
#keep quantity
combined.drop('quantity_group', axis=1, inplace=True)

In [26]:
d = combined
d.groupby(['source', 'source_type', 'source_class']).size() 

source                source_type           source_class
dam                   dam                   surface           656
hand dtw              borehole              groundwater       874
lake                  river/lake            surface           765
machine dbh           borehole              groundwater     11075
other                 other                 unknown           212
rainwater harvesting  rainwater harvesting  surface          2295
river                 river/lake            surface          9612
shallow well          shallow well          groundwater     16824
spring                spring                groundwater     17021
unknown               other                 unknown            66
dtype: int64

In [27]:
#remove source_type
combined.drop('source_type', axis=1, inplace=True)

In [28]:
d = combined
d.groupby(['waterpoint_type', 'waterpoint_type_group']).size() 

waterpoint_type              waterpoint_type_group
cattle trough                cattle trough              116
communal standpipe           communal standpipe       28522
communal standpipe multiple  communal standpipe        6103
dam                          dam                          7
hand pump                    hand pump                17488
improved spring              improved spring            784
other                        other                     6380
dtype: int64

In [29]:
#drop one
combined.drop('waterpoint_type_group', axis=1, inplace=True)

In [30]:
#looking at geographic info now

In [31]:
d = combined
d.groupby(['region', 'region_code', 'district_code']).size()
#regions have unique codes but district codes repeat within regions

region         region_code  district_code
Arusha         2            1                 189
                            2                1206
                            3                 109
                            5                 201
                            6                 310
                            7                1009
               24           30                326
Dar es Salaam  7            1                  93
                            2                 497
                            3                 215
Dodoma         1            0                  23
                            1                 888
                            3                 361
                            4                 347
                            5                 358
                            6                 224
Iringa         11           1                 728
                            2                 530
                            3                 650
        

In [67]:
combined['lga']

0                  ludewa
1               serengeti
2               simanjiro
3                nanyumbu
4                 karagwe
5                  mkinga
6         shinyanga rural
7                  kahama
8            tabora urban
9                 karagwe
10               mkuranga
11               namtumbo
12                  maswa
13                   siha
14                  meatu
15       sumbawanga rural
16                 njombe
17                 njombe
18                ukerewe
19                 ludewa
20                 njombe
21                bariadi
22                 njombe
23                   same
24           kigoma rural
25            moshi rural
26            lindi rural
27                karagwe
28                  rombo
29                   siha
               ...       
59370              kasulu
59371               maswa
59372              urambo
59373              mpanda
59374             bariadi
59375               tanga
59376       singida rural
59377       

In [33]:
#use lga column to create new feature: rural, urban, other
def get_setting(x):
    x=str(x)
    if x.find('Rural') >= 0: return 'rural'
    if x.find('Urban') >= 0: return 'urban'
    return 'other'

In [34]:
combined['rural_urban'] = combined.lga.apply(get_setting)

In [35]:
combined['rural_urban'].value_counts()
#this may not be all that useful as most are in the other column, but will be easy to encode later for random forest

other    48165
rural     9552
urban     1683
Name: rural_urban, dtype: int64

In [36]:
print(type(combined.date_recorded))
print(type(combined.construction_year))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [37]:
#use date time function in pandas
## Change dates to specific format
combined['date_recorded'] = pd.to_datetime(combined['date_recorded'])
combined['construction_year'] = combined.construction_year.map(lambda x: np.nan if x == 0 else x)

In [38]:
combined['operation_year'] = combined.date_recorded.dt.year - combined.construction_year

In [39]:
#print(type(combined.date_recorded))
combined[['operation_year', 'date_recorded', 'construction_year']].head(10)

,operation_year,date_recorded,construction_year
0,12.0,2011-03-14,1999.0
1,3.0,2013-03-06,2010.0
2,4.0,2013-02-25,2009.0
3,27.0,2013-01-28,1986.0
4,NaN,2011-07-13,NaN
5,2.0,2011-03-13,2009.0
6,NaN,2012-10-01,NaN
7,NaN,2012-10-09,NaN
8,NaN,2012-11-03,NaN
9,NaN,2011-08-03,NaN


In [40]:
#create season variable 
#look at monthly counts for reference
combined.date_recorded.dt.month.value_counts().sort_index()

1      6354
2     12402
3     17936
4      3970
5       336
6       346
7      6928
8      3364
9       328
10     5466
11     1349
12      621
Name: date_recorded, dtype: int64

In [41]:
combined['month_recorded'] = combined.date_recorded.dt.month

In [42]:
#Jan and Feb short dry season
#long rains lasts during about March, April and May 
#long dry season lasts throughout June, July, August, September and October 
#During November and December there's another rainy season: the 'short rains'

In [43]:
def get_season (x):
    if x <= 2: return 1 #short dry season 
    elif x <= 5: return 2 #long rains
    elif x <= 10: return 3 #long dry season
    elif x<= 12: return 4 # short rains
    return 
    

In [44]:
#apply function and create new column
combined['season']=combined.month_recorded.apply(get_season)

In [45]:
combined.season.value_counts()

2    22242
1    18756
3    16432
4     1970
Name: season, dtype: int64

In [46]:
#For every categorical response, convert the levels to lower case, in case there is random capitalization
combined.installer = combined.installer.str.lower()
combined.funder = combined.funder.str.lower()
combined.wpt_name = combined.wpt_name.str.lower()
combined.basin = combined.basin.str.lower()
combined.subvillage = combined.subvillage.str.lower()
combined.region = combined.region.str.lower()
combined.lga = combined.lga.str.lower()
combined.ward = combined.ward.str.lower()
combined.management = combined.management.str.lower()
combined.management_group = combined.management_group.str.lower()
combined.scheme_management = combined.scheme_management.str.lower()

In [47]:
#probably not the most elegant but i'm filling NaN with zeros to impute mean coordinates, population, and construction year where missing
combined['longitude'].replace('0', np.nan, inplace=True)
combined['latitude'].replace('0', np.nan, inplace=True)
combined['gps_height'].replace('0', np.nan, inplace=True)
combined['construction_year'].replace('0', np.nan, inplace=True)
combined['operation_year'].replace('0', np.nan, inplace=True)
combined['population'].replace('0', np.nan, inplace=True)

In [48]:
# Compute average longitude and latitude in districts within region to subsitute for zero values found
#impute mean for location coordinates, year, and population using groupby for region and district
#when the district code info isn't available I impute just the regional mean
combined["latitude"].fillna(combined.groupby(['region', 'district_code'])["latitude"].transform("mean"), inplace=True)
combined["longitude"].fillna(combined.groupby(['region', 'district_code'])["longitude"].transform("mean"), inplace=True)
combined["longitude"].fillna(combined.groupby(['region'])["longitude"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined.groupby(['region', 'district_code'])["gps_height"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined.groupby(['region'])["gps_height"].transform("mean"), inplace=True)
combined["gps_height"].fillna(combined["gps_height"].mean(), inplace=True)
combined["construction_year"].fillna(combined["construction_year"].mean(), inplace=True)
combined["population"].fillna(combined.groupby(['region', 'district_code'])["population"].transform("mean"), inplace=True)
combined["population"].fillna(combined.groupby(['region'])["population"].transform("mean"), inplace=True)
combined["population"].fillna(combined["population"].mean(), inplace=True)

In [49]:
combined["funder"].fillna("other", inplace=True)
combined["installer"].fillna("other", inplace=True)
combined["scheme_management"].fillna("other", inplace=True)
combined["subvillage"].fillna("other", inplace=True)

In [50]:
#recalculate operation year
combined['operation_year'] = combined.date_recorded.dt.year - combined.construction_year

In [51]:
#look at missing vals
combined.isnull().sum()

id                       0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
scheme_management        0
construction_year        0
extraction_type          0
extraction_type_class    0
management               0
management_group         0
payment_type             0
water_quality            0
quantity                 0
source                   0
source_class             0
waterpoint_type          0
status_group             0
rural_urban              0
operation_year           0
month_recorded           0
season                   0
dtype: int64

In [52]:
len(list(combined))
#started with 40 columns, now have 34

34

In [53]:
#export cleaned df - have removed a few features and engineered a few
#will clean both test and train dfs this way
#to clean the test set i did this process again with the test set values
pd.DataFrame(combined).to_csv("cleaned_v2.csv")

In [62]:
combined

,id,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quantity,source,source_class,waterpoint_type,status_group,rural_urban,operation_year,month_recorded,season
0,69572,2011-03-14,roman,1390,roman,34.938093,-9.856322e+00,none,0,lake nyasa,...,soft,enough,spring,groundwater,communal standpipe,functional,other,12.000000,3,2
1,8776,2013-03-06,grumeti,1399,grumeti,34.698766,-2.147466e+00,zahanati,0,lake victoria,...,soft,insufficient,rainwater harvesting,surface,communal standpipe,functional,other,3.000000,3,2
2,34310,2013-02-25,lottery club,686,world vision,37.460664,-3.821329e+00,kwa mahundi,0,pangani,...,soft,enough,dam,surface,communal standpipe multiple,functional,other,4.000000,2,1
3,67743,2013-01-28,unicef,263,unicef,38.486161,-1.115530e+01,zahanati ya nanyumbu,0,ruvuma / southern coast,...,soft,dry,machine dbh,groundwater,communal standpipe multiple,non functional,other,27.000000,1,1
4,19728,2011-07-13,action in a,0,artisan,31.130847,-1.825359e+00,shuleni,0,lake victoria,...,soft,seasonal,rainwater harvesting,surface,communal standpipe,functional,other,14.185314,7,3
5,9944,2011-03-13,mkinga distric coun,0,dwe,39.172796,-4.765587e+00,tajiri,0,pangani,...,salty,enough,other,unknown,communal standpipe multiple,functional,other,2.000000,3,2
6,19816,2012-10-01,dwsp,0,dwsp,33.362410,-3.766365e+00,kwa ngomho,0,internal,...,soft,enough,machine dbh,groundwater,hand pump,non functional,rural,15.185314,10,3
7,54551,2012-10-09,rwssp,0,dwe,32.620617,-4.226198e+00,tushirikiane,0,lake tanganyika,...,milky,enough,shallow well,groundwater,hand pump,non functional,other,15.185314,10,3
8,53934,2012-11-03,wateraid,0,water aid,32.711100,-5.146712e+00,kwa ramadhan musa,0,lake tanganyika,...,salty,seasonal,machine dbh,groundwater,hand pump,non functional,urban,15.185314,11,4
9,46144,2011-08-03,isingiro ho,0,artisan,30.626991,-1.257051e+00,kwapeto,0,lake victoria,...,soft,enough,shallow well,groundwater,hand pump,functional,other,14.185314,8,3


In [1]:
def remove_outliers(df, column, min_val, max_val):
    col_values = df[column].values
    df[column] = np.where(np.logical_or(col_values<=min_val, col_values>=max_val), np.NaN, col_values)
    return df
